## Handling multiple sequences

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.7 MB/s eta 0:00:00


In [3]:
# sequence into list of numbers.
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "God is good God is great."
tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

In [4]:
print(ids)

[2643, 2003, 2204, 2643, 2003, 2307, 1012]


In [5]:
# list of number to a tensor.
input_ids = torch.tensor(ids)

In [6]:
model(input_ids)

IndexError: ignored

### This is because the model expect multiple sentences by default

In [8]:
# adding a new dimenstion.
input_ids = torch.tensor([ids])
print("Input IDS:",input_ids)

output = model(input_ids)
print("logits:", output.logits)

Input IDS: tensor([[2643, 2003, 2204, 2643, 2003, 2307, 1012]])
logits: tensor([[-2.3336,  2.4200]], grad_fn=<AddmmBackward0>)


In [9]:
batched_ids = [ids, ids]

In [10]:
input_ids1 = torch.tensor(batched_ids)
print("Input IDS:",input_ids1)

output = model(input_ids1)
print("logits:", output.logits)

Input IDS: tensor([[2643, 2003, 2204, 2643, 2003, 2307, 1012],
        [2643, 2003, 2204, 2643, 2003, 2307, 1012]])
logits: tensor([[-2.3336,  2.4200],
        [-2.3336,  2.4200]], grad_fn=<AddmmBackward0>)


In [11]:
# padding.
sent1_ids = [[200,200,200]]
sent2_ids = [[200,200]]
batched = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

In [12]:
print(model(torch.tensor(sent1_ids)).logits)
print(model(torch.tensor(sent2_ids)).logits)
print(model(torch.tensor(batched)).logits)


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


In [13]:
# We can see we have different logits for same inputs. It is because of attention layer.

In [14]:
# attention mask to get same result.
attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


In [16]:
# try it out 2
seq_1 = "I’ve been waiting for a HuggingFace course my whole life."
seq_2 = "I hate this so much!"
tokens_1 = tokenizer.tokenize(seq_1)
tokens_2 = tokenizer.tokenize(seq_2)
ids1 = tokenizer.convert_tokens_to_ids(tokens_1)
ids2 = tokenizer.convert_tokens_to_ids(tokens_2)

In [20]:
len(ids1)

14

In [27]:
batched1 = [
    [1045, 1521, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012],
[1045, 5223, 2023, 2061, 2172, 999, tokenizer.pad_token_id, tokenizer.pad_token_id, tokenizer.pad_token_id, tokenizer.pad_token_id, tokenizer.pad_token_id, tokenizer.pad_token_id, tokenizer.pad_token_id, tokenizer.pad_token_id],
]

In [28]:
attention_mask = [
    [1, 1, 1,1,1,1,1,1,1,1,1,1,1,1],
    [1, 1, 1,1,1,1,0,0,0,0,0,0,0,0],
]

outputs = model(torch.tensor(batched1), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[-2.5720,  2.6852],
        [ 3.1931, -2.6685]], grad_fn=<AddmmBackward0>)
